In [1]:
# Import the libraries required to estimate the index returns
import pandas as pd
import numpy as np
import spd_matrix as spd
from sklearn.decomposition import PCA
from functools import reduce

# Read in the training dataset provided by Optiver
all_data = pd.read_csv('train.csv')

In [2]:
## This is an attempt to estimate the index time series by performing PCA on the stock data
## It used the weekly index to estimate the covariance matrix - this ensures we remove market noise
## The factor loadings then seem to work as expected and can be thought of as index weights per stock
## Clearly the assumption here is that the index weights remain stable throughout the dataset
## This assumption is not true, so these weights can be thought of as average index weights over the period

# For each week, calculate the stock returns and estimate the covariance matrix - use this covariance matrix in the PCA 
# to then estimate factor loadings of the first factor (likely the market factor) to each of the stocks
end_of_day_data = all_data[all_data['seconds_in_bucket'] == 540]
end_of_week_data = end_of_day_data[end_of_day_data['date_id'] % 5 == 0]
weekly_ts_prices = end_of_week_data.pivot(index='date_id', columns='stock_id', values = 'wap')
weekly_ts_returns = weekly_ts_prices.pct_change(fill_method=None)
weekly_ts_returns = weekly_ts_returns.iloc[1:]

# Once we have the returns we need to z-score the returns per stock to ensure the PCA does not "reward" more volatile ones
weekly_ts_returns_zcore = (weekly_ts_returns - weekly_ts_returns.mean()) / weekly_ts_returns.std(ddof=0)

# Some of the stocks have missing data for some dates/weeks - we will use the pd.cov() function
# This function ignores the missing data and estimates the pairwise correlation which is helpful
cov_matrix = weekly_ts_returns.cov()

# Given the cov() function estimates pairwise correlation ignoring missing data, it might not be PD
# Use the function below to get the nearest PD matrix to the original one before any PCA is done
cov_matrix_spd = spd.nearestPD(cov_matrix)

# Perform the PCA on this correlation matrix to get the factor loadings of the first factor
# This first factor should be the market factor and then we can normalise loadings to get index weights
num_assets = len(weekly_ts_returns.columns)
pca = PCA(n_components=num_assets, svd_solver='full')
pca.fit(cov_matrix_spd)
factor_loadings_cov = pca.components_.T * np.sqrt(pca.explained_variance_)
factor_loadings_mkt_cov = factor_loadings_cov[:, 0]
factor_loadings_mkt_cov = factor_loadings_mkt_cov / sum(factor_loadings_mkt_cov)

idx_wgts = pd.DataFrame(data={'stock_id': list(range(0, num_assets)), 'idx_wgts': factor_loadings_mkt_cov})

In [3]:
## We need to first merge the index weights and replace the index weights for any missing returns data with NaN

# Before we do any calculations, let us remove any unnecessary columns to speed up things
trim_data = all_data.drop(columns=['imbalance_size', 'imbalance_buy_sell_flag', 'matched_size', 'far_price', 'near_price', 'bid_size', 'ask_size', 'target', 'time_id', 'row_id'])

# First, create the return series for each stock as per its respective mid, reference and wap prices
trim_data_with_rets = (
    trim_data
    .assign(
        mid_price = lambda x: 0.5 * (x['bid_price'] + x['ask_price']),
        ref_return = lambda x: x.groupby(['date_id', 'stock_id'])['reference_price'].pct_change(fill_method=None),
        wap_return = lambda x: x.groupby(['date_id', 'stock_id'])['wap'].pct_change(fill_method=None),
        mid_return = lambda x: x.groupby(['date_id', 'stock_id'])['mid_price'].pct_change(fill_method=None)
)
)

stock_rets = trim_data_with_rets.drop(columns=['reference_price', 'bid_price', 'ask_price', 'wap', 'mid_price'])
stock_rets.to_parquet('stock_rets.gzip', compression='gzip')

# Next, let us merge the dataframe with all the data and the one with the index weights
stock_rets_idx_wgts = pd.merge(left=stock_rets, right=idx_wgts, on='stock_id')

# Now replace any index weights where we do not have returns data with NaN (we will use this later to normalise weights)
stock_rets_idx_wgts.loc[stock_rets_idx_wgts['ref_return'].isna(), 'idx_wgts'] = np.nan
stock_rets_idx_wgts.loc[stock_rets_idx_wgts['wap_return'].isna(), 'idx_wgts'] = np.nan
stock_rets_idx_wgts.loc[stock_rets_idx_wgts['mid_return'].isna(), 'idx_wgts'] = np.nan

In [7]:
## Now that we have the index weights, we need to normalise them and then estimate the index returns for reference, mid and wap prices

# This function creates a normalised version fo the weights for any missing returns data
def normalise_wgts(df_sub):
    df_sub['norm_wgts'] = df_sub['idx_wgts'] / float(df_sub['idx_wgts'].sum())
    return df_sub

# Apply the normalise wgts function to our dataframe and remove any unnecessary columns
stock_rets_norm_wgts = stock_rets_idx_wgts.groupby(['date_id', 'seconds_in_bucket']).apply(normalise_wgts, include_groups=False)
stock_rets_norm_wgts.reset_index(inplace=True)
stock_rets_norm_wgts.drop(columns=['level_2', 'idx_wgts'], inplace=True)

# Now we can finally estimate the index returns based on the reference price
all_idx_rets = (
    stock_rets_norm_wgts
    .assign(
        idx_ref_return = lambda x: x['ref_return'] * x['norm_wgts'],
        idx_wap_return = lambda x: x['wap_return'] * x['norm_wgts'],
        idx_mid_return = lambda x: x['mid_return'] * x['norm_wgts']
    ).groupby(['date_id', 'seconds_in_bucket'], as_index=False)
    .agg({'idx_ref_return': 'sum', 'idx_wap_return': 'sum', 'idx_mid_return': 'sum'})
)

all_idx_rets.to_parquet('all_idx_rets.gzip', compression='gzip')

In [ ]:
## The cells below this are ones which were used to test other ways of estimating the index weights
## They did not work as well as the covariance approach using weekly stock data

In [ ]:
## This is an attempt to estimate the index time series by performing PCA on the stock data
## It used the weekly index to estimate the correlation matrix - this ensures we remove market noise
## The factor loadings then seem to work as expected and can be thought of as index weights per stock
## Clearly the assumption here is that the index weights remain stable throughout the dataset
## This assumption is not true, so these weights can be thought of as average index weights over the period

# For each week, calculate the stock returns and estimate the correlation matrix - use this correlation matrix in the PCA 
# to then estimate factor loadings of the first factor (likely the market factor) to each of the stocks
end_of_day_data = all_data[all_data['seconds_in_bucket'] == max(all_data['seconds_in_bucket'])]
end_of_week_data = end_of_day_data[end_of_day_data['date_id'] % 5 == 0]
weekly_ts_prices = end_of_week_data.pivot(index='date_id', columns='stock_id', values = 'wap')
weekly_ts_returns = weekly_ts_prices.pct_change(1)
weekly_ts_returns = weekly_ts_returns.iloc[1:]

# Some of the stocks have missing data for some dates/weeks - we will use the pd.corr() function
# This function ignores the missing data and estimates the pairwise correlation which is helpful
corr_matrix = weekly_ts_returns.corr()

# Given the corr() function estimates pairwise correlation ignoring missing data, it might not be PD
# Use the function below to get the nearest PD matrix to the original one before any PCA is done
corr_matrix_spd = spd.nearestPD(corr_matrix)

# Perform the PCA on this correlation matrix to get the factor loadings of the first factor
# This first factor should be the market factor and then we can normalise loadings to get index weights
num_assets = len(weekly_ts_returns.columns)
pca = PCA(n_components=num_assets, svd_solver='full')
pca.fit(corr_matrix_spd)
factor_loadings_corr = pca.components_.T * np.sqrt(pca.explained_variance_)
factor_loadings_mkt_corr = factor_loadings_corr[:, 0]
factor_loadings_mkt_corr = factor_loadings_mkt_corr / sum(factor_loadings_mkt_corr)

In [ ]:
## This was an attempt to estimate the index time series by performing PCA on the stock data
## It used the 10s tick data to estimate the correlation matrix - this tick data will have a lot of noise
## The factor loadings did not work as expected due to this noise

# For each date, calculate the stock returns and estimate the correlation matrix - use this correlation matrix in the PCA 
# to then estimate factor loadings of the first factor (likely the market factor) to each of the stocks
max_dates = max(all_data['date_id']) + 1
max_assets = max(all_data['stock_id']) + 1
daily_loadings_mkt = np.empty((max_dates, max_assets,))
daily_loadings_mkt[:] = np.nan

for date_id in range(max(all_data['date_id']) + 1):
    date_data = all_data[all_data['date_id'] == date_id]
    # Pivot the dataframe to create a wap time series for all the stocks
    date_ts_prices = date_data.pivot(index='time_id', columns='stock_id', values = 'wap')
    # Check for any empty columns and remove them
    date_ts_prices.dropna(how='all', axis=1, inplace=True)   
    stock_ids = sorted(date_ts_prices.columns.unique())
    num_assets = len(date_ts_prices.columns)
    date_ts_returns = date_ts_prices.pct_change(1)
    date_ts_returns = date_ts_returns.iloc[1:]
    corr_matrix = date_ts_returns.corr()
    pca = PCA(n_components=num_assets, svd_solver='full')
    pca.fit(corr_matrix)
    factor_loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    factor_loadings_mkt = factor_loadings[:, 0]
    factor_loadings_mkt = factor_loadings_mkt / sum(factor_loadings_mkt)
    daily_loadings_mkt[date_id, stock_ids] = factor_loadings_mkt